In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import rules.v02_fixing_Arp2_3 as rul
from rules.enzymes import *
import itertools

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {
    Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, Dia1, Dia2, Ena_Vasp,
    IRSp53, LIMK, PAK, PIP2, PKD, Profilin, ROCK, Rac1, RhoA, SSH, Thymosin,
    WASP, WAVE
}

rules_map = {}
orig_rules = rul.build_rules()
rules_map[base_name] = orig_rules

# knockdowns
for nodes in sorted([{a, b} for a, b in itertools.combinations_with_replacement(sorted(all_nodes), 2)]):
# for nodes in [{WASP, PKD}, {WASP, ROCK}]:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']

    for node in nodes:
        graph.remove_edges_from([(node, target)
                                 for target in list(graph[node])])
        rules = re.sub('1:\s*' + node + '\*\s*=.+', '', rules)
        rules = rules + '1: ' + node + '* = False \n'

    rules_map[base_name + "_del_" + "_".join(sorted(nodes))] = {
        "graph": graph,
        "rules": rules
    }

sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

In [63]:
import re
import rules.v03_oscillations as rul
from rules.enzymes import *
import itertools

base_name = re.sub(".+\.", "", rul.__name__)

all_nodes = {
    Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, Dia1, Dia2, Ena_Vasp,
    IRSp53, LIMK, PAK, PIP2, PKD, Profilin, ROCK, Rac1, RhoA, SSH, Thymosin,
    WASP, WAVE
}

rules_map = {}
orig_rules = rul.build_rules()
rules_map[base_name] = orig_rules

# # knockdowns
# for nodes in sorted([{a, b} for a, b in itertools.combinations_with_replacement(sorted(all_nodes), 2)]):
for nodes in [{WAVE}, {Dia1}, {Dia2}, {WASP}, {Cortactin}, {Ena_Vasp}, {PAK}, {Profilin}, {CP}]:
# for nodes in []:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']

    for node in nodes:
        graph.remove_edges_from([(node, target)
                                 for target in list(graph[node])])
        rules = re.sub('1:\s*' + node + '\*\s*=.+', '', rules)
        rules = rules + '1: ' + node + '* = False \n'

    rules_map[base_name + "_del_" + "_".join(sorted(nodes))] = {
        "graph": graph,
        "rules": rules
    }

sorted(rules_map.keys())

# print(rules_map['v03_oscillations']['rules'])

['v03_oscillations',
 'v03_oscillations_del_CP',
 'v03_oscillations_del_Cortactin',
 'v03_oscillations_del_Dia1',
 'v03_oscillations_del_Dia2',
 'v03_oscillations_del_Ena_Vasp',
 'v03_oscillations_del_PAK',
 'v03_oscillations_del_Profilin',
 'v03_oscillations_del_WASP',
 'v03_oscillations_del_WAVE']

In [ ]:
### run selected rules

%run params.ipynb

for name, rules in sorted(rules_map.items()):
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 8
    PARAMS['chunks'] = 16
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST], [Rac1, RhoA], [WASP, Thymosin, Cofilin]]
    PARAMS['rewrite'] = True
    
#     %run "01. structural analysis.ipynb"        
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

====== v03_oscillations
Running boolean simulations on 16 CPUs
Chunk created 5
Chunk created 4
Chunk created 1
Chunk created 0
Chunk created 9
Chunk created 10
Chunk created 11
Chunk created 8
Chunk created 3
Chunk created 2
Chunk created 6
Chunk created 15
Chunk created 14
Chunk created 7
Chunk created 13
Chunk created 12
Removing previous simulations
Saving chunks to data file
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity
====== v03_oscillations_del_CP
Running boolean simulations on 16 CPUs
Chunk created 0
Chunk created 15
Chunk created 5
Chunk created 2
Chunk created 1
Chunk created 10
Chunk created 8
Chunk created 11
Chunk created 4
Chunk created 6
Chunk created 3
Chunk created 9
Chunk created 12
Chunk created 14
Chunk created 7
Chunk created 13
Removing previous simulations
Saving chunks to data file
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity
====== v03_oscillations_del_Cortactin

In [65]:
# build summary for selected rules map
import pandas as pd
import numpy as np

def color_delta(src):
    base = src['base']
    if base == 0:
        colors = ['black'] * len(src)
    else:
        colors = [ 'red' if s > 1.2 else 'blue' if s < 0.8 else 'black' for s in src / base]    
    return [ 'color: {}'.format(c)  for c in colors]

cases = sorted(rules_map.keys())
rep = pd.read_csv("reports/"+cases[0]+"/weighted_activity.csv", index_col=0)
rep.columns = ["base"]
for case in cases[1:]:
    file_name = "reports/"+case+"/weighted_activity.csv"
    if os.path.isfile(file_name):
        name = re.sub(base_name+"_del_", "", case)
        rep[name] = pd.read_csv("reports/"+case+"/weighted_activity.csv", index_col=0).iloc[:, 0].rename(name)

rep['mean'] = rep.mean(axis=1)
rep.to_csv("reports/"+cases[0]+"_summary.csv")

styler = rep.style.apply(color_delta, axis=1)

with open('reports/'+cases[0]+'_summary.html', 'w') as fo:
    fo.write(styler.render())
    
styler

In [ ]:
# build double knockdown summary for selected rules map
import pandas as pd
import numpy as np
import os.path

def color_delta(src, base):
    if src.name[1] == 'BR':
        b = base[0]
    else:
        b = base[1]
        
    colors = [ 'gray' if pd.isnull(s) else 'red' if s > 0 else 'blue' if s < 0 else 'black' for s in src - b]  
    return [ 'color: {}'.format(c)  for c in colors]


na = 'n/a'
BR = 'BR'
ST = 'ST'
midx = pd.MultiIndex.from_product([[na] + sorted(all_nodes), [BR, ST]])
report = pd.DataFrame(index=midx, columns=[na] + sorted(all_nodes), data=None)

cases = sorted(rules_map.keys())

#base
rep = pd.read_csv("reports/"+cases[0]+"/weighted_activity.csv", index_col=0)
base_BR = rep['activity, %']['Actin_BR']
base_ST = rep['activity, %']['Actin_ST']
report[na][na] = (base_BR, base_ST)

node_re = '|'.join(all_nodes)

for case in cases[1:]:
    file_name = "reports/"+case+"/weighted_activity.csv"   
    if os.path.isfile(file_name):               
        name = re.sub(base_name+"_del_", "", case)    
        match = re.match("("+node_re+")(_("+node_re+"))?", name) 
        a, b = match.group(1, 3)
        if b is None:
            b = na
        
        rep = pd.read_csv(file_name, index_col=0)
        values = (rep['activity, %']['Actin_BR'], rep['activity, %']['Actin_ST'])
        report[a][b] = values
        report[b][a] = values        


report.to_csv("reports/"+cases[0]+"_double_knockdown_summary.csv")       
styler = report.style.apply(color_delta, axis=1, base=(base_BR, base_ST))

additional_style= """
<style type="text/css">
table { border-collapse: collapse; }
thead { border-bottom: solid 1px black; }
tbody > tr:nth-child(2n + 3) { border-top: solid 1px black; }
tbody > tr:nth-child(4n - 2), tbody > tr:nth-child(4n - 3) { background-color: #eee; }
.data, .col_heading {
    border-left: solid 1px black;
    padding-left: 0.3rem;
    padding-right: 0.3rem;
    text-align: center;
}
</style>
"""

with open('reports/'+cases[0]+'_double_knockdown_summary.html', 'w') as fo:
    fo.write("<html><body>"+additional_style + styler.render()+"</body></html>")

# TODO: 
#  +/- 20% difference is important
    
styler